# Chapter 7 - Modelling Workflows as pipelines

## Pipelines
* We can think of a workflow simply as some sequence of substeps
* In our book thus far we've looked at a _Place Order_ workflow, comprised of validating an order, pricing it, shipping, etc
* A statement was made earlier in the book that business' real value is the transformations they make on inputs (data); well isn't that exactly what we're doing here?
* Each pipe ought to be small, and do only one thing. We can then chain these together to form a larger pipeline
    * A name for this is transformation-oriented-programming

## Workflow Inputs
* Inputs should always be domain objects (this makes sense too)

### Commands as Inputs
* At the very start of the book, we spoke about commands creating data, so what makes sense here is that our command is actually the very first input. 
* For example, instead of an `UnvalidatedOrder` being our input, perhaps the command `PlaceOrder` should be.

In [14]:
// Giving it a shite name so I can use the nice name below
type PlaceOrderData = {
    OrderForm: Map<String, String>;
    Timestamp: DateTime;
    UserId: string;
}

### Sharing Common Structures with Generics
* We've spoken before about structural generics
* Why redefine shared shapes when we can lean on this

In [15]:
type Command<'T> = {
    Data: 'T;
    TimeStamp: DateTime;
    UserId: string;
}

type UnvalidatedOrder =  { OrderForm: Map<string, string>; }

type PlaceOrder = Command<UnvalidatedOrder>

### Combining Commands
* There are times where commands for a bounded context will be sent on the same input channel
    * Let's be real this is 100% of the time
* We need a way of encapsulating this

In [16]:
type OrderCommand = 
  | Place of PlaceOrder
  | Change of string
  | Cancel of string

## Modelling an Order as a set of States

* We can often feel tempted to create massive records to capture all of the possible data that a record will have
* We ought to fight this urge, and think about what data a record might contain at any point in time
    * I personally like to think of this as expressing the life cycle of something, via its contents
* A huge advantage about getting granular is that we can really focus only on what matters in that particular state
* Another advantage is easy extensibility. In the example below you'll be able to see that to add a new order type / state, there's only one place to do it

In [17]:
type ValidatedOrder = {
    OrderId: string;
    CustomerInfo: Map<string, string> // Using a bit of a hack hear to save redefining things
    ShippingAddress: Map<string, string>
}

// I'll concede I *really* wish F# would let us have intersects or whatever
type PricedOrder = {
    OrderId: string;
    CustomerInfo: Map<string, string>
    ShippingAddress: Map<string, string>
    Price: bigint // something fun like in cents
}

type Order = 
  | Validated of ValidatedOrder
  | Priced of PricedOrder

## State Machines
* As a little CS refresher: _a device which can be in one of a set number of stable conditions depending on its previous condition and on the present values of its inputs._
* We have states, and we can only transition to and from states via defined methods
* The book has some nice examples, of shopping cards or email addresses (the latter already discussed)

### Why Would we use State Machines?
1. Each state has different behaviour
2. All states are explicitly documented
3. We can use state machines as a design tool to force us to think about every possible state

I want to take a moment to talk about Scott's summary of edge cases.

> A common cause of errors in design is that certain edge cases are not handled. [...] What should happen if we try to validate a validated email address?

### Implementing State Machines in F#
* We touched on it above, but definitely (again) don't be building gigantic record types to capture all of your data. Let the _state_ do the talking
* We can take all of these states, and wrap them in some common type, like we did with `Order` above
* And again, we can then implement a single handler that will do the work in consuming this common state

In [20]:
// A very crude state transition where a validated order becomes priced, and any priced order is already priced!
let priceOrder order = 
    match order with
    | Validated v -> {OrderId = v.OrderId; CustomerInfo = v.CustomerInfo; ShippingAddress = v.ShippingAddress; Price = 100 |> bigint}
    | Priced p -> p


Stopped due to error


Error: input.fsx (4,24)-(4,28) parse error Unexpected keyword 'with' in binding. Expected incomplete structured construct at or before this point or other token.
input.fsx (5,14)-(5,15) typecheck error The value or constructor 'p' is not defined.

* The code in the book for this section demonstrates actual transitions, but it's quite eas